In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import Column

In [2]:
sc=SparkContext(appName="pg_dataframe") 
spark=SparkSession.builder.appName('pg_dataframe').master("Yct201811021847").getOrCreate()

In [3]:
def create_df_from_postgres():
    df=spark.read.format('jdbc').options(
        url='jdbc:postgresql://localhost:5432/northwind',
        dbtable='public.orders',
        user='postgres',
        password='iamroot'
    ).load()
    return df

In [4]:
data=create_df_from_postgres()

In [6]:
data.show(2)

+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+------------------+--------+----------+--------------+-----------+
|orderid|customerid|employeeid| orderdate|requireddate|shippeddate|shipvia|freight|            shipname|       shipaddress|shipcity|shipregion|shippostalcode|shipcountry|
+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+------------------+--------+----------+--------------+-----------+
|  10248|     VINET|         5|1996-07-04|  1996-08-01| 1996-07-16|      3|  32.38|Vins et alcools C...|59 rue de l'Abbaye|   Reims|      null|         51100|     France|
|  10249|     TOMSP|         6|1996-07-05|  1996-08-16| 1996-07-10|      1|  11.61|  Toms Spezialitäten|     Luisenstr. 48| Münster|      null|         44087|    Germany|
+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+------------------+--------+----------+--

In [7]:
type(spark)

pyspark.sql.session.SparkSession

In [8]:
data.printSchema()

root
 |-- orderid: short (nullable = true)
 |-- customerid: string (nullable = true)
 |-- employeeid: short (nullable = true)
 |-- orderdate: date (nullable = true)
 |-- requireddate: date (nullable = true)
 |-- shippeddate: date (nullable = true)
 |-- shipvia: short (nullable = true)
 |-- freight: float (nullable = true)
 |-- shipname: string (nullable = true)
 |-- shipaddress: string (nullable = true)
 |-- shipcity: string (nullable = true)
 |-- shipregion: string (nullable = true)
 |-- shippostalcode: string (nullable = true)
 |-- shipcountry: string (nullable = true)



In [10]:
type(data)

pyspark.sql.dataframe.DataFrame

In [11]:
data.summary().show()

+-------+-----------------+----------+------------------+------------------+------------------+--------------------+--------------------+--------+----------+------------------+-----------+
|summary|          orderid|customerid|        employeeid|           shipvia|           freight|            shipname|         shipaddress|shipcity|shipregion|    shippostalcode|shipcountry|
+-------+-----------------+----------+------------------+------------------+------------------+--------------------+--------------------+--------+----------+------------------+-----------+
|  count|              830|       830|               830|               830|               830|                 830|                 830|     830|       323|               811|        830|
|   mean|          10662.5|      null| 4.403614457831325|2.0072289156626506|   78.244204877492|                null|                null|    null|      null|39975.067357512955|       null|
| stddev|239.7446558319914|      null|2.499647540082388

In [26]:
data.describe(['freight']).show()

+-------+------------------+
|summary|           freight|
+-------+------------------+
|  count|               830|
|   mean|   78.244204877492|
| stddev|116.77929406259489|
|    min|              0.02|
|    max|           1007.64|
+-------+------------------+



In [27]:
cust_country=data.cube(['customerid','freight','shipcountry'])

In [28]:
type(cust_country)

pyspark.sql.group.GroupedData

In [29]:
cust_country.count().show()

+----------+-------+-----------+-----+
|customerid|freight|shipcountry|count|
+----------+-------+-----------+-----+
|     HANAR|  65.83|     Brazil|    1|
|     QUICK| 229.24|    Germany|    1|
|     HUNGC|    0.2|        USA|    1|
|     VICTE|   4.81|     France|    1|
|     BLONP|  59.14|     France|    1|
|     OLDWO| 170.97|        USA|    1|
|      null| 147.26|       null|    1|
|     GODOS| 107.83|       null|    1|
|     LONEP|   0.56|       null|    1|
|      null|  47.42|       null|    1|
|     SAVEA|  89.16|       null|    1|
|      null| 370.61|       null|    1|
|      null| 148.61|  Venezuela|    1|
|     BSBEV|   4.87|       null|    1|
|      null| 105.81|       null|    1|
|     QUEDE|   9.53|       null|    1|
|     VICTE|  22.11|       null|    1|
|      null|   2.71|       null|    1|
|      null|  52.51|       null|    1|
|     BERGS|  32.37|       null|    1|
+----------+-------+-----------+-----+
only showing top 20 rows



In [30]:
cust_country.avg().show()

+----------+-------+-----------+------------+---------------+------------+-------------------+
|customerid|freight|shipcountry|avg(orderid)|avg(employeeid)|avg(shipvia)|       avg(freight)|
+----------+-------+-----------+------------+---------------+------------+-------------------+
|     HANAR|  65.83|     Brazil|     10250.0|            4.0|         2.0|  65.83000183105469|
|     QUICK| 229.24|    Germany|     10286.0|            8.0|         3.0| 229.24000549316406|
|     HUNGC|    0.2|        USA|     10415.0|            3.0|         1.0|0.20000000298023224|
|     VICTE|   4.81|     France|     10478.0|            2.0|         3.0|  4.809999942779541|
|     BLONP|  59.14|     France|     10584.0|            4.0|         1.0|  59.13999938964844|
|     OLDWO| 170.97|        USA|     10855.0|            3.0|         1.0| 170.97000122070312|
|      null| 147.26|       null|     10294.0|            4.0|         2.0| 147.25999450683594|
|     GODOS| 107.83|       null|     10303.0|     